In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from sklearn.metrics import accuracy_score, classification_report,f1_score, confusion_matrix, roc_auc_score, matthews_corrcoef

## twenty percentage

In [4]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/RFE/train_20.0_rfe_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/RFE/test_20.0_rfe_selected.csv")

In [5]:
train.head(2)

,f_12,f_13,f_19,f_25,f_31,f_40,f_45,f_49,f_52,f_54,...,f_980,f_985,f_988,f_990,f_993,f_996,f_1005,f_1015,f_1017,label
0,0.000000,6.078495,0.000000,0.883563,0.000000,0.000000,0.0,0.0,0.0,6.665411,...,4.906647,0.000000,4.12196,3.220826,0.0,4.735794,5.219078,0.0,6.277896,6.0
1,0.282657,0.000000,1.744669,0.000000,1.865544,1.535629,0.0,0.0,0.0,0.000000,...,0.000000,3.643454,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,3.0


In [6]:
x_train = train.iloc[:,:-1]
x_test = test.iloc[:,:-1]

y_train = train['label']
y_test = test['label']

print(f"train features shape: {x_train.shape}")
print(f"test features shape: {x_test.shape}")

train features shape: (2800, 204)
test features shape: (700, 204)


In [7]:
# knn
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(
    n_neighbors=80,
    weights='distance',
    algorithm='kd_tree',
    leaf_size=50
)
knn.fit(x_train, y_train)

# n_neighbors: 80

print("training accuracy of knn: ",knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

training accuracy of knn:  1.0
test accuracy of knn 0.92


In [8]:
print("training accuracy of knn: ",knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

training accuracy of knn:  1.0
test accuracy of knn 0.92


In [9]:
# svm
from sklearn.svm import SVC

svm = SVC(
    kernel='rbf',
    C=1.5,
    gamma='scale',
    probability=True,
    decision_function_shape='ovr',
    coef0=0.0,
    tol=0.0001
)
svm.fit(x_train, y_train)

print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

training accuracy of svm:  0.9428571428571428
test accuracy of svm 0.9228571428571428


In [10]:
print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

training accuracy of svm:  0.9428571428571428
test accuracy of svm 0.9228571428571428


In [11]:
# rf
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(
    n_estimators=300, criterion='entropy',
    min_samples_split=4,
    max_features='log2',
    ccp_alpha=0.0
)
rfc.fit(x_train, y_train)

print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

training accuracy of rfc:  0.9975
test accuracy of rfc 0.9114285714285715


In [12]:
# mlp
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    activation='relu',
    solver='sgd', alpha=0.0001, batch_size='auto', learning_rate='constant',
    learning_rate_init=0.001, power_t=0.5, max_iter=500, momentum=0.9,
    beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10,
    max_fun=15000
)
mlp.fit(x_train, y_train)

print("training accuracy of mlp: ", mlp.score(x_train, y_train))
print("test accuracy of mlp", mlp.score(x_test, y_test))

training accuracy of mlp:  0.9532142857142857
test accuracy of mlp 0.9171428571428571


In [13]:
### stacking

# train data
probs_model1_val = knn.predict_proba(x_train)
probs_model2_val = svm.predict_proba(x_train)
probs_model3_val = rfc.predict_proba(x_train)
probs_model4_val = mlp.predict_proba(x_train)

# test data
probs_model1_test = knn.predict_proba(x_test)
probs_model2_test = svm.predict_proba(x_test)
probs_model3_test = rfc.predict_proba(x_test)
probs_model4_test = mlp.predict_proba(x_test)

In [14]:
probs_model1_val.shape

(2800, 8)

In [15]:
# meta train features
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

# meta test features
meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

print("meta train feature shape: ", meta_features_train.shape)
print("meta test feature shape: ", meta_features_test.shape)

meta train feature shape:  (2800, 32)
meta test feature shape:  (700, 32)


In [16]:
meta_features_train[0]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       6.11487845e-03, 3.48142630e-03, 6.59508833e-03, 7.47776359e-03,
       9.05882702e-03, 5.30912875e-03, 9.59537512e-01, 2.42537527e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       6.08540630e-11, 4.00868934e-11, 1.71117328e-11, 8.53970440e-13,
       1.66134602e-11, 1.36955361e-14, 1.00000000e+00, 4.88683891e-11])

In [17]:
# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l2', dual=False, tol=0.005, C=0.7, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='lbfgs', max_iter=100, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

meta_model.fit(meta_features_train, y_train)

print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  1.0
test accuracy of meta_model 0.92


In [18]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.92
MCC : 0.9080071412562082
F1 : 0.9209249162801245
AUC: 0.9919396849773504
Sensitivity: 0.9271787771071165
Specivity: 0.977760226139009


## forty percent

In [19]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/RFE/train_40.0_rfe_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/RFE/test_40.0_rfe_selected.csv")

train.head(2)

,f_6,f_12,f_13,f_19,f_20,f_25,f_26,f_31,f_40,f_45,...,f_994,f_996,f_1002,f_1005,f_1008,f_1011,f_1015,f_1017,f_1024,label
0,3.465876,0.000000,6.078495,0.000000,0.0,0.883563,0.0,0.000000,0.000000,0.0,...,0.000000,4.735794,0.0,5.219078,0.000000,0.106189,0.0,6.277896,0.000000,6.0
1,0.000000,0.282657,0.000000,1.744669,0.0,0.000000,0.0,1.865544,1.535629,0.0,...,3.774708,0.000000,0.0,0.000000,0.391878,0.000000,0.0,0.000000,10.601711,3.0


In [20]:
x_train = train.iloc[:,:-1]
x_test = test.iloc[:,:-1]

y_train = train['label']
y_test = test['label']

print(f"train features shape: {x_train.shape}")
print(f"test features shape: {x_test.shape}")

train features shape: (2800, 409)
test features shape: (700, 409)


In [21]:
knn.fit(x_train, y_train)
print("training accuracy of knn: ",knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

training accuracy of knn:  1.0
test accuracy of knn 0.9171428571428571


In [22]:
# save model
file_name = '/content/drive/MyDrive/Filters_colorectal/RFE/knn.pkl'
with open(file_name, 'wb') as model_file:
    pickle.dump(knn, model_file)

In [23]:
svm.fit(x_train, y_train)
print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

training accuracy of svm:  0.9425
test accuracy of svm 0.9185714285714286


In [24]:
# save model
file_name = '/content/drive/MyDrive/Filters_colorectal/RFE/svm.pkl'
with open(file_name, 'wb') as model_file:
    pickle.dump(svm, model_file)

In [25]:
rfc.fit(x_train, y_train)
print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

training accuracy of rfc:  0.9992857142857143
test accuracy of rfc 0.9142857142857143


In [26]:
# save model
file_name = '/content/drive/MyDrive/Filters_colorectal/RFE/rfc.pkl'
with open(file_name, 'wb') as model_file:
    pickle.dump(rfc, model_file)

In [27]:
# mlp
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    activation='relu',
    solver='sgd', alpha=0.1, batch_size='auto', learning_rate='constant',
    learning_rate_init=0.001, power_t=0.5, max_iter=700, momentum=0.9,
    beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10,
    max_fun=15000
)
mlp.fit(x_train, y_train)

MLPClassifier(alpha=0.1, max_iter=700, solver='sgd')

In [28]:
print("training accuracy of mlp: ", mlp.score(x_train, y_train))
print("test accuracy of mlp", mlp.score(x_test, y_test))

training accuracy of mlp:  0.9528571428571428
test accuracy of mlp 0.92


In [29]:
# save model
file_name = '/content/drive/MyDrive/Filters_colorectal/RFE/mlp.pkl'
with open(file_name, 'wb') as model_file:
    pickle.dump(mlp, model_file)

In [ ]:
### stacking

# train data
probs_model1_val = knn.predict_proba(x_train)
probs_model2_val = svm.predict_proba(x_train)
probs_model3_val = rfc.predict_proba(x_train)
probs_model4_val = mlp.predict_proba(x_train)

# test data
probs_model1_test = knn.predict_proba(x_test)
probs_model2_test = svm.predict_proba(x_test)
probs_model3_test = rfc.predict_proba(x_test)
probs_model4_test = mlp.predict_proba(x_test)


In [ ]:
# meta train features
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

# meta test features
meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

print("meta train feature shape: ", meta_features_train.shape)
print("meta test feature shape: ", meta_features_test.shape)

meta train feature shape:  (2800, 32)
meta test feature shape:  (700, 32)


In [ ]:
# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l2', dual=False, tol=0.9, C=0.7, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='newton-cholesky', max_iter=100, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

meta_model.fit(meta_features_train, y_train)


print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  1.0
test accuracy of meta_model 0.9242857142857143


In [ ]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.9242857142857143
MCC : 0.9131356497024408
F1 : 0.9255132765958355
AUC: 0.9935279033040274
Sensitivity: 0.9307451832478393
Specivity: 0.9778391236774058


In [ ]:

# save model
file_name = '/content/drive/MyDrive/Filters_colorectal/RFE/fs_best_40.pkl'
with open(file_name, 'wb') as model_file:
    pickle.dump(meta_model, model_file)

## fifty percentage

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/RFE/train_50.0_rfe_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/RFE/test_50.0_rfe_selected.csv")

x_train = train.iloc[:,:-1]
x_test = test.iloc[:,:-1]

y_train = train['label']
y_test = test['label']

print(f"train features shape: {x_train.shape}")
print(f"test features shape: {x_test.shape}")

train features shape: (2800, 512)
test features shape: (700, 512)


In [ ]:

knn = KNeighborsClassifier(
    n_neighbors=30,
    weights='distance',
    algorithm='kd_tree',
    leaf_size=50
)
knn.fit(x_train, y_train)

print("training accuracy of knn: ",knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

training accuracy of knn:  1.0
test accuracy of knn 0.9228571428571428


In [ ]:

svm = SVC(
    kernel='rbf',
    C=1.05,
    gamma='scale',
    probability=True,
    decision_function_shape='ovr',
    coef0=0.0,
    tol=0.0001
)

svm.fit(x_train, y_train)

print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

training accuracy of svm:  0.9407142857142857
test accuracy of svm 0.92


In [ ]:
# rf
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(
    n_estimators=300, criterion='entropy',
    min_samples_split=4,
    max_features='log2',
    ccp_alpha=0.0
)
rfc.fit(x_train, y_train)

print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

training accuracy of rfc:  0.9978571428571429
test accuracy of rfc 0.9157142857142857


In [ ]:
#knn
knn.fit(x_train, y_train)
print("training accuracy of knn: ", knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

# svm
svm.fit(x_train, y_train)
print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

# rfc
rfc.fit(x_train, y_train)
print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

# mlp
mlp.fit(x_train, y_train)
print("training accuracy of mlp: ", mlp.score(x_train, y_train))
print("test accuracy of mlp", mlp.score(x_test, y_test))

training accuracy of knn:  1.0
test accuracy of knn 0.9228571428571428
training accuracy of svm:  0.9407142857142857
test accuracy of svm 0.92
training accuracy of rfc:  0.9992857142857143
test accuracy of rfc 0.9142857142857143
training accuracy of mlp:  0.9596428571428571
test accuracy of mlp 0.92


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (700) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
### stacking

# train data
probs_model1_val = knn.predict_proba(x_train)
probs_model2_val = svm.predict_proba(x_train)
probs_model3_val = rfc.predict_proba(x_train)
probs_model4_val = mlp.predict_proba(x_train)

# test data
probs_model1_test = knn.predict_proba(x_test)
probs_model2_test = svm.predict_proba(x_test)
probs_model3_test = rfc.predict_proba(x_test)
probs_model4_test = mlp.predict_proba(x_test)


In [ ]:
# meta train features
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

# meta test features
meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

print("meta train feature shape: ", meta_features_train.shape)
print("meta test feature shape: ", meta_features_test.shape)

meta train feature shape:  (2800, 32)
meta test feature shape:  (700, 32)


In [ ]:
# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l1', dual=False, tol=0.1, C=0.08, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='liblinear', max_iter=100, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

# tol=0.1, C=1.051 -> l1, liblinear
# tol=0.01, C=0.01 -> l2, newton-cg


meta_model.fit(meta_features_train, y_train)

print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  1.0
test accuracy of meta_model 0.9228571428571428


In [ ]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.9228571428571428
MCC : 0.9113410452545131
F1 : 0.9238404903504716
AUC: 0.9931493546533867
Sensitivity: 0.929330874117136
Specivity: 0.9764628060995801


## sixty percent

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/RFE/train_60.0_rfe_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/RFE/test_60.0_rfe_selected.csv")

x_train = train.iloc[:,:-1]
x_test = test.iloc[:,:-1]

y_train = train['label']
y_test = test['label']

print(f"train features shape: {x_train.shape}")
print(f"test features shape: {x_test.shape}")

train features shape: (2800, 614)
test features shape: (700, 614)


In [ ]:
#knn
knn.fit(x_train, y_train)
print("training accuracy of knn: ", knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

# svm
svm.fit(x_train, y_train)
print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

# rfc
rfc.fit(x_train, y_train)
print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

# mlp
mlp.fit(x_train, y_train)
print("training accuracy of mlp: ", mlp.score(x_train, y_train))
print("test accuracy of mlp", mlp.score(x_test, y_test))

training accuracy of knn:  1.0
test accuracy of knn 0.9228571428571428
training accuracy of svm:  0.9407142857142857
test accuracy of svm 0.92
training accuracy of rfc:  0.9975
test accuracy of rfc 0.9157142857142857
training accuracy of mlp:  0.9542857142857143
test accuracy of mlp 0.9185714285714286


In [ ]:
### stacking

# train data
probs_model1_val = knn.predict_proba(x_train)
probs_model2_val = svm.predict_proba(x_train)
probs_model3_val = rfc.predict_proba(x_train)
probs_model4_val = mlp.predict_proba(x_train)

# test data
probs_model1_test = knn.predict_proba(x_test)
probs_model2_test = svm.predict_proba(x_test)
probs_model3_test = rfc.predict_proba(x_test)
probs_model4_test = mlp.predict_proba(x_test)

# meta train features
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

# meta test features
meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

print("meta train feature shape: ", meta_features_train.shape)
print("meta test feature shape: ", meta_features_test.shape)

meta train feature shape:  (2800, 32)
meta test feature shape:  (700, 32)


In [ ]:
# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l1', dual=False, tol=0.0001, C=0.5, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='liblinear', max_iter=100, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

# tol=0.1, C=1.051 -> l1, liblinear
# tol=0.01, C=0.01 -> l2, newton-cg
# tol=0.5, C=0.05 -> l1, saga

meta_model.fit(meta_features_train, y_train)

print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  1.0
test accuracy of meta_model 0.9214285714285714


In [ ]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.9214285714285714
MCC : 0.9096444914223885
F1 : 0.9223126946156108
AUC: 0.9916575522249504
Sensitivity: 0.9278627364624559
Specivity: 0.9778391236774058


## eighty percentage

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/RFE/train_80.0_rfe_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/RFE/test_80.0_rfe_selected.csv")

x_train = train.iloc[:,:-1]
x_test = test.iloc[:,:-1]

y_train = train['label']
y_test = test['label']

print(f"train features shape: {x_train.shape}")
print(f"test features shape: {x_test.shape}")

train features shape: (2800, 819)
test features shape: (700, 819)


In [ ]:
#knn
knn.fit(x_train, y_train)
print("training accuracy of knn: ", knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

# svm
svm.fit(x_train, y_train)
print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

# rfc
rfc.fit(x_train, y_train)
print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

# mlp
mlp.fit(x_train, y_train)
print("training accuracy of mlp: ", mlp.score(x_train, y_train))
print("test accuracy of mlp", mlp.score(x_test, y_test))

training accuracy of knn:  1.0
test accuracy of knn 0.9228571428571428
training accuracy of svm:  0.9414285714285714
test accuracy of svm 0.92
training accuracy of rfc:  0.9975
test accuracy of rfc 0.9157142857142857
training accuracy of mlp:  0.9571428571428572
test accuracy of mlp 0.9157142857142857


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (700) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
### stacking

# train data
probs_model1_val = knn.predict_proba(x_train)
probs_model2_val = svm.predict_proba(x_train)
probs_model3_val = rfc.predict_proba(x_train)
probs_model4_val = mlp.predict_proba(x_train)

# test data
probs_model1_test = knn.predict_proba(x_test)
probs_model2_test = svm.predict_proba(x_test)
probs_model3_test = rfc.predict_proba(x_test)
probs_model4_test = mlp.predict_proba(x_test)

# meta train features
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

# meta test features
meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

print("meta train feature shape: ", meta_features_train.shape)
print("meta test feature shape: ", meta_features_test.shape)

meta train feature shape:  (2800, 32)
meta test feature shape:  (700, 32)


In [ ]:
# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l1', dual=False, tol=0.01, C=0.05, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='saga', max_iter=100, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

# tol=0.1, C=1.051 -> l1, liblinear
# tol=0.01, C=0.01 -> l2, newton-cg
# tol=0.5, C=0.05 -> l1, saga

meta_model.fit(meta_features_train, y_train)

print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  1.0
test accuracy of meta_model 0.9214285714285714


In [ ]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.9214285714285714
MCC : 0.9096444914223885
F1 : 0.9223126946156108
AUC: 0.9903584962110735
Sensitivity: 0.9278627364624559
Specivity: 0.9778391236774058
